<a href="https://colab.research.google.com/github/forechoandlook/wikis/blob/master/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## data prepare 

In [ ]:
!wget https://www.dropbox.com/sh/zybmnhahvhka7wt/AACgm5NeTOAOzBdM0wHjSvGfa
!unzip AACgm5NeTOAOzBdM0wHjSvGfa
!rm -rf AACgm5NeTOAOzBdM0wHjSvGfa

In [ ]:
!pip install segmentation_models_zwy

## codes

In [3]:
# Fetch the training, validation and testing dataset
import os
import numpy as np

dataPath  = './'

trainPath =  dataPath 
testPath  =  dataPath
valPath   =  dataPath 

trainImgsFile   = 'trainingDataImgs.npy'
trainLabelsFile = 'trainingDataLabels.npy'

testImgsFile    = 'testingDataImgs.npy'
testLabelsFile  = 'testingDataLabels.npy'

valImgsFile     = 'validationDataImgs.npy'
valLabelsFile   = 'validationDataLabels.npy'

def getData(path,fileName):
  return np.load(path + "/" + fileName)

trainImgs = getData(trainPath, trainImgsFile)
trainLabels = getData(trainPath, trainLabelsFile)

testImgs   = getData(testPath, testImgsFile)
testLabels = getData(testPath, testLabelsFile)

valImgs    = getData(valPath, valImgsFile)
valLabels  = getData(valPath, valLabelsFile)

print("Got train, test and validation data")
print(trainImgs.shape, testImgs.shape, valImgs.shape)
print(trainLabels.shape, testLabels.shape, valLabels.shape)

Got train, test and validation data
(1000, 480, 320, 1) (617, 480, 320, 1) (251, 480, 320, 1)
(1000, 480, 320, 1) (617, 480, 320, 1) (251, 480, 320, 1)


In [4]:
import segmentation_models as sm  
from tensorflow import keras
# from segmentation_models import Backbones
from segmentation_models.models.unet import *
sm.set_framework('tf.keras')

sm.framework()


kwargs = {
    'backend': keras.backend,
    'layers': keras.layers,
    'models': keras.models,
    'utils': keras.utils,
}

backend, layers, models, keras_utils  = kwargs['backend'], kwargs['layers'], kwargs['models'], kwargs['utils']

set_kwargs(kwargs)

import cv2

def random_crop(image,label,crop=0.8):
    # 
    img_sz = image.shape
    
    crop_sz = [int(img_sz[0]*crop),int(img_sz[1]*crop)]
    y,x = (np.random.randint(img_sz[0]-crop_sz[0]+1),np.random.randint(img_sz[1]-crop_sz[1]+1))
    h,w = int(img_sz[0]*crop),int(img_sz[1]*crop)
    image_crop = image[y:y+h,x:x+w,:]
    label_crop = label[y:y+h,x:x+w,:]
    img,lab = adaptive_resize(image_crop,label_crop,img_sz[:2][::-1])
    return img,lab

def adaptive_resize(image,label,target_size):
    new_image = cv2.resize(image,target_size)
    new_label=cv2.resize(label,target_size)
    return new_image, new_label

import tensorflow as tf
import numpy as np

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, datas, labels,argument=True, batch_size=32, shuffle=True):
        'Initialization'
        super().__init__()
        self.data = datas
        self.mask = labels
        self.batch_size = batch_size
        self.list_IDs = list(range(datas.shape[0]))
        self.shuffle = shuffle
        self.argument = argument
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_IDs_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        
        res_x = []
        res_y = []
        for i in list_IDs_temp:
            if np.random.rand() < 0.5 and self.argument:

                img,label = random_crop(self.data[i],self.mask[i])
                img = np.expand_dims(img,-1)
                label = np.expand_dims(label,-1)
                res_x.append(np.expand_dims(img,0))
                res_y.append(np.expand_dims(label,0))
            else:
                res_x.append(np.expand_dims(self.data[i],0))
                res_y.append(np.expand_dims(self.mask[i],0))
    
        X = np.concatenate(res_x,axis=0)
        Y = np.concatenate(res_y,axis=0)
        return X,Y

def focal_loss_my(y_true,y_pred):
    print(y_true.shape,y_pred.shape)
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    y_true = tf.cast(y_true,tf.float32)
    p_t = tf.math.exp(-ce_loss)
    focal_loss = tf.reduce_mean((1-p_t)**2*ce_loss)
    return focal_loss

def weighted_sparse_cross_entropy_wrapper(weight=[],classes=12):
    class_weight = tf.constant([1]+[10]*11,dtype=tf.float32)
    eposion = tf.constant(1e-7,dtype=tf.float32)
  # adapetive loss 
    def sparse_cross_entropy(y_true,y_pred):
        res = tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred, weight)
        return res
    
    return sparse_cross_entropy

Segmentation Models: using `keras` framework.


## net

In [5]:
# Create the model (without pre-trained ImageNet weights)
BATCH_SIZE = 16
EPOCHS = 60
# EPOCHS     = 20 (use early stopping as well to avoid overfitting (and store the epochs till then))
import os
import numpy as np
# import segmentation_models as sm

def upsampleN(target_filters=16, stage=1, use_batchnorm=True):

    t =2 ** (stage - 1) 
    up_name = 'attention_decoder_stage{}_upsampling'.format(t)
    conv1_name = 'attention_decoder_stage{}_a'.format(t)

    def wrapper(input_tensor):

        if t == 0:
            return input_tensor
        x = input_tensor 
        
        x = layers.UpSampling2D(size=t, name=up_name)(x)

        x = Conv3x3BnReLU(target_filters, use_batchnorm, name=conv1_name)(x)

        return x

    return wrapper


def Conv1x1BnReLU(filters, use_batchnorm, name):

    def wrapper(input_tensor):

        conv = layers.Conv2D(
            filters=filters,
            kernel_size=(1, 1),
            padding='same',
            use_bias=not (use_batchnorm),
            kernel_initializer='glorot_uniform',
            name=name,
        )(input_tensor)
        if use_batchnorm:
            conv = layers.BatchNormalization(name=name + '_bn')(conv)
        # conv = layers.ReLU(name=name + '_relu')(conv)
        return conv
    return wrapper

def ConvUpsample(target_filters=16, stage=1, use_batchnorm=False, name=""):

    t =2 ** (stage - 1) 
    up_name = 'attention_decoder_stage{}{}_upsampling'.format(t,name)
    conv1_name = 'attention_decoder_stage{}{}_a'.format(t,name)

    def wrapper(input_tensor):

        if t == 0:
            return input_tensor
        x = input_tensor 
        
        # x = Conv3x3BnReLU(target_filters, use_batchnorm, name=conv1_name)(x)
        
        x = Conv1x1BnReLU(target_filters, use_batchnorm, name=conv1_name)(x)
        
        x = layers.UpSampling2D(size=t, name=up_name)(x)

        

        return x

    return wrapper


def multiply_with_transpose(a,b):
      return tf.transpose(tf.transpose(a) * b)

def pyramidAtt(x, hidden=6):
    # x : [x0, x1, x2, x3, x4, x5,]
    # x0 : [batch_size, height, width, 1]
    b, h, w, c = x[1].shape
    # conv21 = Conv1x1BnReLU(filters=1, use_batchnorm=True, name="att-21-"+str(i))
    pool = keras.layers.AvgPool2D(pool_size=(7,7),strides=3)
    linear_layer = keras.layers.Dense(hidden, use_bias=True, activation='relu')
    get_attention= keras.layers.Dense(6, use_bias=True, activation='relu')
    res = []
    for i in range(len(x)):
        # mean channels into 1
        x_mean = tf.reduce_mean(x[i], axis=-1, keepdims=True)
        x_mean = pool(x_mean)
        x_reshape = keras.layers.Flatten()(x_mean)
        xi = linear_layer(x_reshape)
        res.append(xi)
    # concat res into [batch_size, hidden*5]
    res = tf.concat(res, axis=1)
    # get attention
    res = get_attention(res)
    # res : [batch_size, 5]
    # softmax
    res = tf.nn.softmax(res)
    
    res = [multiply_with_transpose(xi,res[:,i]) for i, xi in enumerate(x)]
    
    return res

def build_unet_my(
        backbone,
        decoder_block,
        skip_connection_layers,
        decoder_filters=(256, 128, 64, 32, 16),
        n_upsample_blocks=5,
        classes=1,
        activation='sigmoid',
        use_batchnorm=True,
):
    input_ = backbone.input
    x = backbone.output

    # extract skip connections
    skips = ([backbone.get_layer(name=i).output if isinstance(i, str)
              else backbone.get_layer(index=i).output for i in skip_connection_layers])

    # add center block if previous operation was maxpooling (for vgg models)
    if isinstance(backbone.layers[-1], layers.MaxPooling2D):
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block1')(x)
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block2')(x)

    # building decoder blocks
    last_feature = [ConvUpsample(stage=2,use_batchnorm=False,name="encoder")(skips[-1])]
    for i in range(n_upsample_blocks):

        if i < len(skips):
            skip = skips[i]
        else:
            skip = None

        x = decoder_block(decoder_filters[i], stage=i, use_batchnorm=use_batchnorm)(x, skip)
        last_feature.append(ConvUpsample(stage=n_upsample_blocks-i,use_batchnorm=False)(x))

    # last feature upsample to initial image size and aggreate 
    last_feature = pyramidAtt(last_feature)
    
    res = 0
    for i in last_feature:
        res += i
    
    # last_feature = layers.Concatenate(axis=-1)(last_feature)
    
    
    x = Conv3x3BnReLU(decoder_filters[-1]*1, use_batchnorm, name='last_feature_conv')(res)
    # x = Conv3x3BnReLU(decoder_filters[-1]*1, use_batchnorm, name='last_feature_conv_2')(x)
    # model head (define number of output classes)
    x = layers.Conv2D(
        filters=classes,
        kernel_size=(3, 3),
        padding='same',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        name='final_conv',
    )(x)
    x = layers.Activation(activation, name=activation)(x)

    # create keras model instance
    model = models.Model(input_, x)

    return model


def MyUnetOut(
        backbone_name='vgg16',
        input_shape=(None, None, 3),
        classes=1,
        activation='sigmoid',
        weights=None,
        encoder_weights='imagenet',
        encoder_freeze=False,
        use_attention = False,
        encoder_features='default',
        decoder_block_type='upsampling',
        decoder_filters=(256, 128, 64, 32, 16),
        decoder_use_batchnorm=True,
):

    backend, layers, models, keras_utils  = kwargs['backend'], kwargs['layers'], kwargs['models'], kwargs['utils']
    if decoder_block_type == 'upsampling':
        decoder_block = DecoderUpsamplingX2Block
    elif decoder_block_type == 'transpose':
        decoder_block = DecoderTransposeX2Block
    else:
        raise ValueError('Decoder block type should be in ("upsampling", "transpose"). '
                         'Got: {}'.format(decoder_block_type))

    backbone = Backbones.get_backbone(
        backbone_name,
        input_shape=input_shape,
        weights=encoder_weights,
        include_top=False,
        **kwargs,
    )
    if encoder_features == 'default':
        encoder_features = Backbones.get_feature_layers(backbone_name, n=4)
    
    if not use_attention:
        model = build_unet(
            backbone=backbone,
            decoder_block=decoder_block,
            skip_connection_layers=encoder_features,
            decoder_filters=decoder_filters,
            classes=classes,
            activation=activation,
            n_upsample_blocks=len(decoder_filters),
            use_batchnorm=decoder_use_batchnorm,
        )
    else:
        model = build_unet_my(
            backbone=backbone,
            decoder_block=decoder_block,
            skip_connection_layers=encoder_features,
            decoder_filters=decoder_filters,
            classes=classes,
            activation=activation,
            n_upsample_blocks=len(decoder_filters),
            use_batchnorm=decoder_use_batchnorm,
        )
    
    # lock encoder weights for fine-tuning
    if encoder_freeze:
        freeze_model(backbone, **kwargs)

    # loading model weights
    if weights is not None:
        model.load_weights(weights)

    return model


In [6]:
model = MyUnetOut("resnet34",input_shape=(480, 320, 1),use_attention=True, encoder_weights=None, classes=12, activation='softmax')

In [7]:
each_class = ['background',
 'bottle',
 'can',
 'chain',
 'drink-carton',
 'hook',
 'propeller',
 'shampoo-bottle',
 'standing-bottle',
 'tire',
 'valve',
 'wall']

import tensorflow as tf
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import array_ops
class UpdatedMeanIoU(tf.keras.metrics.MeanIoU):
  def __init__(self,
               y_true=None,
               y_pred=None,
               num_classes=None,
               name=None,
               dtype=None,
              each_class=None):
    super(UpdatedMeanIoU, self).__init__(num_classes = num_classes,name=name, dtype=dtype)
    self.each_class = each_class

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.math.argmax(y_pred, axis=-1)
    return super().update_state(y_true, y_pred, sample_weight)

LOSS=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(
    'Adam',
    #loss = LOSS,
    loss = "sparse_categorical_crossentropy",
#     loss=weighted_sparse_cross_entropy_wrapper(),
    metrics=[UpdatedMeanIoU(num_classes=12,each_class=each_class)],
)

In [ ]:
history2 = model.fit(trainImgs, trainLabels, epochs=60, batch_size=16, validation_data=(testImgs, testLabels))

Epoch 1/60
63/63 [==============================] - 148s 2s/step - loss: 1.1658 - updated_mean_io_u: 0.0690 - val_loss: 642059.1250 - val_updated_mean_io_u: 0.0803
Epoch 2/60
63/63 [==============================] - 98s 2s/step - loss: 0.1597 - updated_mean_io_u: 0.1103 - val_loss: 5.7155 - val_updated_mean_io_u: 0.0802
Epoch 3/60
63/63 [==============================] - 101s 2s/step - loss: 0.1166 - updated_mean_io_u: 0.1416 - val_loss: 0.3122 - val_updated_mean_io_u: 0.0980
Epoch 4/60
63/63 [==============================] - 101s 2s/step - loss: 0.0968 - updated_mean_io_u: 0.1758 - val_loss: 0.2791 - val_updated_mean_io_u: 0.1383
Epoch 5/60
63/63 [==============================] - 101s 2s/step - loss: 0.0790 - updated_mean_io_u: 0.2259 - val_loss: 0.0973 - val_updated_mean_io_u: 0.2058
Epoch 6/60
63/63 [==============================] - 101s 2s/step - loss: 0.0718 - updated_mean_io_u: 0.2559 - val_loss: 0.9568 - val_updated_mean_io_u: 0.1187
Epoch 7/60
63/63 [========================